# Install


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

import data_import
from enrich import process_transactions

import model

load_dotenv("../user.env")

True

# Fetch


In [50]:
import ast
import os
from loguru import logger

from pathlib import Path

from services.nordigen import config, account_api
import utils

In [67]:
def get_requisition_id():
    requisition_ids = ast.literal_eval(os.environ.get("requisition_ids"))
    logger.info(f"found {len(requisition_ids)} requisition_ids")
    
    if not requisition_ids:
        raise ValueError("no requisition ids found")
    
    if not isinstance(requisition_ids, list):
        raise TypeError("invalid type found")

    if len(requisition_ids)>1:
        logger.warning("More than 1 requisition id found")
        raise NotImplementedError("unhandled >1 requisition")

    requisition_id = requisition_ids[0]
    return requisition_id
# get first requisition id


output_filepath = await account_api.get_latest_data(requisition_id=get_requisition_id())

2024-01-18 12:54:27.354 | INFO     | __main__:get_requisition_id:3 - found 1 requisition_ids
2024-01-18 12:54:27.355 | INFO     | services.nordigen.account_api:get_latest_data:226 - today's data exists


In [68]:
output_filepath

PosixPath('../data/fetched/2024-01-18_data.json')

In [70]:
latest_data = data_import.load_user_data(filepath = output_filepath)

# Tidy up output


In [ ]:
def get_account_details(account_ids) -> dict[str, dict]:
    account_details = {}
    for account_id in account_ids:
        account = client.account_api(id=account_id)
        # parent
        details = account.get_details()
        account_details[account_id] = details
    return account_details


# def get_account_transactions(account_ids) -> dict[str, dict]:
#     transactions = {}
#     for account_id in account_ids:
#         account = client.account_api(id=account_id)
#         # parent
#         data = account.get_transactions()
#         transactions[account_id] = data
#     return transactions

# def get_account_balances(account_ids):
#     balances = {}
#     for account_id in account_ids:
#         account = client.account_api(id=account_id)
#         # parent
#         data = account.get_balances()
#         balances[account_id] = data
#     return balances


def get_account_metadata(account_ids):
    metadata = {}
    for account_id in account_ids:
        account = client.account_api(id=account_id)
        # parent
        data = account.get_metadata()
        # missing metadata key in api
        metadata[account_id] = {"metadata": data}
    return metadata

In [ ]:
# dynamic
account_transactions = get_account_transactions(account_ids)
balances = get_account_balances(account_ids)
# static
account_metadata = get_account_metadata(account_ids)
account_details = get_account_details(account_ids)

In [ ]:
_data = utils.read_json(api_output_filepath)
_data.get("data")[1][0]

# Parse


In [ ]:
user_data = data_import.load_user_data()

In [ ]:
user_data.requisition.id

# Enrich


In [ ]:
user_data.transactions = process_transactions.process_transactions(
    user_data.transactions
)

# Query


In [ ]:
import pandas as pd

In [ ]:
records = user_data.query_transactions_by_transaction_code(
    "FASTER PAYMENT DEBIT REVERSAL"
)